In [25]:
import pandas as pd
import os

In [26]:
path = os.path.join('OriginalData', 'education.csv')
data = pd.read_csv(path)
data_backup = data.copy()

In [27]:
# Show NaN values
data.isnull().sum()

user_id                  0
school_name              0
degree               35842
fields_of_study      19838
start_year_month    134373
end_year_month      134985
dtype: int64

In [28]:
# Too many NaNs, drop start_year_month and end_year_month
data.drop(['start_year_month', 'end_year_month'], axis=1, inplace=True)
data

,user_id,school_name,degree,fields_of_study
0,0,Anadolu Üniversitesi,NaN,NaN
1,0,Adıyaman Üniversitesi,NaN,NaN
2,0,Fırat Üniversitesi,NaN,NaN
3,1,Fırat Üniversitesi,Yüksek Lisans,Yazılım Mühendisliği
4,1,Fırat Üniversitesi,Lisans,Yazılım Mühendisliği
...,...,...,...,...
142570,66272,Halmstad University,NaN,Computer Science
142571,66272,Bogazici University,NaN,Chemistry
142572,66273,Istanbul Technical University,Bachelor's degree,Electronic and Communication Engineering
142573,66273,Bogazici University,Master's degree,Electrical and Electronics Engineering


In [29]:
# Lowercase all rows in the dataframe
data = data.applymap(lambda s:s.lower() if type(s) == str else s)
data

,user_id,school_name,degree,fields_of_study
0,0,anadolu üniversitesi,NaN,NaN
1,0,adıyaman üniversitesi,NaN,NaN
2,0,fırat üniversitesi,NaN,NaN
3,1,fırat üniversitesi,yüksek lisans,yazılım mühendisliği
4,1,fırat üniversitesi,lisans,yazılım mühendisliği
...,...,...,...,...
142570,66272,halmstad university,NaN,computer science
142571,66272,bogazici university,NaN,chemistry
142572,66273,istanbul technical university,bachelor's degree,electronic and communication engineering
142573,66273,bogazici university,master's degree,electrical and electronics engineering


In [30]:
# add a column for each user, with the value of the amount of rows
# that the user has in the dataset
data['education_count'] = data.groupby('user_id')['user_id'].transform('count')

In [31]:
# remove trailing and leading spaces
data['fields_of_study'] = data['fields_of_study'].str.strip()

In [32]:
# Has went to university
data['attended_university'] = data['school_name'].apply(lambda x: 1 if 'üniversite' in x or 'university' in x or 'uni' in x or 'üni' in x or 'univ' in x else 0)
data

,user_id,school_name,degree,fields_of_study,education_count,attended_university
0,0,anadolu üniversitesi,NaN,NaN,3,1
1,0,adıyaman üniversitesi,NaN,NaN,3,1
2,0,fırat üniversitesi,NaN,NaN,3,1
3,1,fırat üniversitesi,yüksek lisans,yazılım mühendisliği,2,1
4,1,fırat üniversitesi,lisans,yazılım mühendisliği,2,1
...,...,...,...,...,...,...
142570,66272,halmstad university,NaN,computer science,3,1
142571,66272,bogazici university,NaN,chemistry,3,1
142572,66273,istanbul technical university,bachelor's degree,electronic and communication engineering,3,1
142573,66273,bogazici university,master's degree,electrical and electronics engineering,3,1


In [33]:
from fuzzywuzzy import fuzz
from collections import defaultdict

def group_strings(strings, reference_strings, similarity_threshold):
    groups = defaultdict(list)
    for i, string1 in enumerate(reference_strings):
        for j, string2 in enumerate(strings):
            similarity = fuzz.token_set_ratio(string1, string2)
            if similarity >= similarity_threshold:
                groups[string1].append(string2)
    return groups

In [34]:
# Engineering, IT, science
engineering_groups = ["Mühendis", "Mühendislik", "Müh.", "Mühendisliği", "Engineering", "Engi", "Bilgisayar", "Bilişim", "Bilim", "Teknoloji", "Meka", "Makine", "Machine", "Veri", "inşaat", "science", "sciences", "Web"]

# Business, Marketing etc.
business_group = ["Finance", "Marketing", "Business", "İş", "Pazarlama", "İşletme", ]

# Lower all the groups
engineering_groups = [x.lower() for x in engineering_groups]
business_group = [x.lower() for x in business_group]


skill_groups = engineering_groups + business_group
skill_groups

['mühendis',
 'mühendislik',
 'müh.',
 'mühendisliği',
 'engineering',
 'engi',
 'bilgisayar',
 'bilişim',
 'bilim',
 'teknoloji',
 'meka',
 'makine',
 'machine',
 'veri',
 'inşaat',
 'science',
 'sciences',
 'web',
 'finance',
 'marketing',
 'business',
 'i̇ş',
 'pazarlama',
 'i̇şletme']

In [35]:
skills = data['fields_of_study'].unique().tolist()

In [36]:
skill_groups = group_strings(skills, skill_groups, 50)

In [37]:
# Combine it skills
engineering_fields = []
for key, value in skill_groups.items():
    if key in engineering_groups:
        engineering_fields += value
it_skills = list(set(engineering_fields))

# Combine business skills
business_fields = []
for key, value in skill_groups.items():
    if key in business_group:
        business_fields += value
business_skills = list(set(business_fields))


In [38]:
# Check for any similarities
for a in it_skills:
    if a in business_skills:
        print(a)

marka yönetimi
bilgisayar mühendisliği(i̇ngilizce/tezli)
müzik öğretmenliği
engineering of business management
elektronik mühendisliği-%30 i̇ngilizce
bilgisayar teknolojileri / i̇nternet ve ağ teknolojiler
i̇şletme mühendisliği
i̇nşaat teknolojisi
matematik mühendisliği (i̇ng)
i̇ngilizce elektrik ve elektronik mühendisliği
makine (i̇.ö)
i̇nşaat teknolojileri
fen / science
bilgiişlem
institute of social sciences, business administration, mba
mekatronik mühendisliği (i̇ngilizce)
makine mühendisliği(i̇ngilizce eğitim dili)
i̇çerik editörlüğü
computer science, finance and business analytics
i̇zmir ,faculty of engineering ,department of food engineering
bilgisayar ve öğretim teknolojileri öğretmenliği (i̇ngilizce)
noromarketing
mathematik
nitelikli bilişim uzmanı yetiştirme kursu(i̇şkur)
deniz ulaştırma ve i̇şletme mühendisliği
computer & i̇nformation science engineering
matematik mühendisliği(i̇ngilizce)
işletme mühendisliği
business administration in german, bachelor of science (bsc)
gemi

In [39]:
# Remove NaN values
it_skills = [x for x in it_skills if str(x) != 'nan']
business_skills = [x for x in business_skills if str(x) != 'nan']


In [40]:
for a in business_skills:
    if a in it_skills:
        # Remove from business skills
        business_skills.remove(a)

In [41]:
# View
business_skills

['deniz ulaştırma ve i̇şletme',
 'business administration & international trade (double major)',
 'i̇şletme yönetimi',
 'reklamcılık ve marka i̇letişimi',
 'pharmacy',
 'brand school - marketing communications',
 'i̇ktisadi ve i̇dari bilimler fakültesi',
 'business analytics',
 'management faculty, business administration',
 'international business / marketing',
 'management faculty, business administration (turkish)',
 'pazarlama iletişimi',
 'i̇ngilizce i̇ktisat',
 'business information systems',
 'i̇ngilizce(b1-b2) eğitimi',
 'çalışma  ekonomisi ve endüstriyel i̇lişkiler',
 'i̇statistik ve bilgisayar',
 'c sınıfı i̇sg uzmanı',
 'i̇nternet proğramcılığı',
 "bachelor's  degree, business  administration  and  management",
 'siyaset bilimi ve uluslararası i̇lişkiler',
 'turizm işletmeciliği',
 'business and information management',
 'bilgisayar mühendisliği(i̇ngilizce/tezli)',
 'müzik öğretmenliği',
 'i̇şletme, yönetim ve operasyon',
 'information technology and business management',
 '

In [42]:
it_skills

['web design',
 'lisans-endüstri mühendisliği',
 'information technologies / web design and programming',
 'fen bilimleri/sayısal',
 'elektrik teknolojileri',
 'mining engineering',
 'web programming and web designs',
 'bilişim teknolojileri web programlama alanı',
 'sosyal bilimler enstitüsü, mis - yönetim bilişim sistemleri',
 'mechanical engineering - machine theory and dynamics',
 'mathematics & sciences',
 'bilgisayar ve bilisim muhendisligi',
 'chemical engineering',
 'ekonomi',
 'computer science and software engineering(csse)',
 'computer science&engineering(100% english)',
 'marka yönetimi',
 'yönetim bilişim sistemleri - mis (almanca)',
 'bilgisayar mühendisliği(i̇ngilizce/tezli)',
 'bilgisayar öğretim tekolojileri eğitimi',
 'müzik öğretmenliği',
 'electronics, telecommunications and radio engineering',
 'hesaplamalı bilim ve mühendislik',
 'faculty of mines, earth sciences/geological engineering',
 'compuer science',
 'bilgisayar progrmcılığı',
 'elektronik haberleşme mühen

In [43]:
# Rename fields_of_study values to it or business, or None
data['fields_of_study'] = data['fields_of_study'].apply(lambda x: 'it' if x in it_skills else 'business' if x in business_skills else "other")
data


,user_id,school_name,degree,fields_of_study,education_count,attended_university
0,0,anadolu üniversitesi,NaN,other,3,1
1,0,adıyaman üniversitesi,NaN,other,3,1
2,0,fırat üniversitesi,NaN,other,3,1
3,1,fırat üniversitesi,yüksek lisans,it,2,1
4,1,fırat üniversitesi,lisans,it,2,1
...,...,...,...,...,...,...
142570,66272,halmstad university,NaN,it,3,1
142571,66272,bogazici university,NaN,it,3,1
142572,66273,istanbul technical university,bachelor's degree,it,3,1
142573,66273,bogazici university,master's degree,it,3,1


In [44]:
# Cast degree column into str
data['degree'] = data['degree'].astype(str)

In [45]:
# If degree name contains any of these, then it is a master's degree
master_degree = ['master', 'm.s', 'm.s.', 'm.s.c', 'm.s.c.', 'm.s.c.e', 'm.s.c.e.', 'm.s.e', 'm.s.e.', "yüksek", "doktora", "msc"]

# If degree name contains any of these, then it is a bachelor's degree
bachelor_degree = ['bachelor', 'b.s', 'b.s.', 'b.s.c', 'b.s.c.', 'b.s.c.e', 'b.s.c.e.', 'b.s.e', 'b.s.e.', 'lisans', 'bsc', "associate", "b.e."]

# Apply the above functions to the degree name column
data['degree'] = data['degree'].apply(lambda x: 'master' if any(item in x for item in master_degree) else 'bachelor' if any(item in x for item in bachelor_degree) else "other")

In [46]:
data

,user_id,school_name,degree,fields_of_study,education_count,attended_university
0,0,anadolu üniversitesi,other,other,3,1
1,0,adıyaman üniversitesi,other,other,3,1
2,0,fırat üniversitesi,other,other,3,1
3,1,fırat üniversitesi,master,it,2,1
4,1,fırat üniversitesi,bachelor,it,2,1
...,...,...,...,...,...,...
142570,66272,halmstad university,other,it,3,1
142571,66272,bogazici university,other,it,3,1
142572,66273,istanbul technical university,bachelor,it,3,1
142573,66273,bogazici university,master,it,3,1


In [47]:
# Save the data
data.to_csv("PreparedData/education.csv", index=False)